In [1]:
import torch
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
%matplotlib inline

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from fastai.text import *
import glob

In [19]:
predictionDB = pd.read_csv("../data/processed/predictionDB.csv",lineterminator='\n')
embeddings = np.load('./../data/processed/embeddings/0003349d47ee039e4600f75f0b7c893aa63e481e.npy')


embeddings = [None]*len(predictionDB)
i=0
for np_name in glob.glob('./../data/processed/embeddings/*.np[yz]'):
    embeddings[i] = np.load(np_name)
    i = i + 1

embeddings
print(len(predictionDB),i)

62917

In [40]:
embeddings2 = pd.Series( (v for v in embeddings) )
embeddings2

0        [-0.004385929, -0.044906683, -0.012293607, -0....
1        [-0.04903131, 0.0013047587, 0.0012576148, -0.0...
2        [-0.036767572, -0.002614243, -0.06437786, 0.04...
3        [-0.010170586, 0.05198802, -0.023644155, -0.07...
4        [-0.105558835, -0.030839775, -0.05386243, 0.00...
                               ...                        
62912    [0.0070811766, 0.005670004, -0.033401966, 0.08...
62913    [-0.014596588, -0.02113724, 0.03371435, 0.0676...
62914    [0.004544533, -0.02132213, -0.03033029, 0.1139...
62915    [-0.076373085, 0.00037320715, -0.058804136, 0....
62916    [-0.04915071, -0.025180638, -0.03297432, -0.00...
Length: 62917, dtype: object

In [41]:
from sklearn.model_selection import train_test_split

#data = embeddings
labels = predictionDB["inc_complexity"]

data_train, data_test, labels_train, labels_test = train_test_split(embeddings2, labels, test_size=0.20, random_state=42)


In [42]:
type(embeddings2)

pandas.core.series.Series

In [82]:
class commits_dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X.index)
    
    def __getitem__(self, index):
        return self.X[index],self.y[index]

In [83]:
commits_dataset_train = commits_dataset(X=data_train,y=labels_train)
commits_dataset_test = commits_dataset(X=data_test,y=labels_test)

In [84]:
train_loader = DataLoader(dataset=commits_dataset_train, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=commits_dataset_test, batch_size=32, shuffle=False)
#dls = DataLoaders(train_loader,valid_loader)


In [85]:
#predictionDB["is_valid"] = np.zeros(len(predictionDB))
#for i in range(len(predictionDB["is_valid"])):
#    predictionDB["is_valid"][i] = 1 if random.random()<0.2 else 0 

In [86]:
#from fastai.text.all import *
#dls = TextDataLoaders.from_df(predictionDB, text_col='COMMIT_MESSAGE', label_col='inc_complexity', valid_col='is_valid')
#dls.show_batch(max_n=3)

In [87]:
# Multilayer perceptron
class MultilayerPerceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(1503, 250, bias=True) 
        self.lin2 = nn.Linear(250, 120, bias=True)
        self.lin3 = nn.Linear(120, 1, bias=True)

    def forward(self, xb):
        x = xb.float()
        #x = xb.view(250, -1)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return self.lin3(x)

In [88]:
#mlp_learner = Learner(data=data, model=MultilayerPerceptron(), loss_func=nn.CrossEntropyLoss(),metrics=accuracy)
#mlp_learner.fine_tune(20)

In [89]:
model = MultilayerPerceptron()
print(model)

MultilayerPerceptron(
  (lin1): Linear(in_features=1503, out_features=250, bias=True)
  (lin2): Linear(in_features=250, out_features=120, bias=True)
  (lin3): Linear(in_features=120, out_features=1, bias=True)
)


In [90]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

In [91]:
mean_train_losses = []
mean_valid_losses = []
valid_acc_list = []
epochs = 15

for epoch in range(epochs):
    model.train()
    
    train_losses = []
    valid_losses = []
    for i, (embeddings, labels) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        outputs = model(embeddings)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if (i * 128) % (128 * 100) == 0:
            print(f'{i * 128} / 50000')
            
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (embeddings, labels) in enumerate(valid_loader):
            outputs = model(embeddings)
            loss = loss_fn(outputs, labels)
            
            valid_losses.append(loss.item())
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
    mean_train_losses.append(np.mean(train_losses))
    mean_valid_losses.append(np.mean(valid_losses))
    
    accuracy = 100*correct/total
    valid_acc_list.append(accuracy)
    print('epoch : {}, train loss : {:.4f}, valid loss : {:.4f}, valid acc : {:.2f}%'\
         .format(epoch+1, np.mean(train_losses), np.mean(valid_losses), accuracy))

KeyError: 42280